[Signals- & Systems](https://github.com/spatialaudio/signals-and-systems-exercises),
[University of Rostock](https://www.uni-rostock.de/en/),
[Institute of Communications Engineering](https://www.int.uni-rostock.de/),
Prof. [Sascha Spors](https://orcid.org/0000-0001-7225-9992),
[Frank Schultz](https://orcid.org/0000-0002-3010-0294),
Till Rettberg,
[CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import MaxNLocator
import numpy as np
from scipy import signal

# Analysis of Discrete-Time LTI-System

Evaluate and plot the

* impulse response
* step response
* frequency response (magnitude, phase, group delay)
* zero/pole mapping, i.e. z-plane plot

of a discrete-time LTI system given as z-transfer function

\begin{equation}
H(z) = \frac{\sum\limits_{n=0}^{N} b_n z^{-n}}{\sum\limits_{m=0}^{M} a_m z^{-m}}=\frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + ...}{a_0 + a_1 z^{-1} + a_2 z^{-2} + ...}
\end{equation}

using `scipy.signal` routines for discrete-time domain signal processing. The coefficients $b_0, b_1,b_2,...,b_N$ are stored in vector/array b, the coefficients $a_0, a_1,a_2,...,a_M$ are stored in vector/array a.

Most often $H(z)$ is normalized such that $a_0=1$.

Note that `signal.dlti` handling has a known issue (https://dsp.stackexchange.com/questions/40312/why-does-pythons-scipy-signal-dimpulse-introduce-delay-in-impulse-response).
For workaround, the **user must ensure** that **a and b** have the **same length** by suitable zero padding.
This is tedious for a long FIR.
Here this might be acceptable, since this notebook is intended for didactical purpose.
If you need analysis of long FIRs, you might want to implement a performant DFT-based handling on your own, until this might be fixed in `scipy.signal`.

Another issue that is not solved yet in the quasi-standard packages is the zplane plot.
Thus, we make our own implementation here.
It is not fully satisfactory yet, when systems exhibit multiple poles/zeros at exactly the same position.
The current handling numbers each occurring pole/zero, for which then multiple numbering at about same location indicates multiple zeros or poles.
Again, for larger number of poles/and or zeros this will become a messy plot.

Used abbreviations:

DFT...discrete Fourier transform

DTFT...discrete-time Fourier transform

FIR...finite impulse response

IIR...infinite impulse response

LTI...linear time-invariant

ROC...region of convergence for z-transform.
Since we aim at causal impulse responses, ROC must be $|z|>\mathrm{max}(|z_{\infty}|)$.
Thus, if ROC (grey area in subsequent plots) includes the unit circle (along which DTFT is defined), this implies that all poles are within the unit circle and by that that the causal system response is stable.

## Evaluating and Plotting Routine

Let us first collect the handling for calculating and plotting the relevant system parameters. At the moment the handling is not consistent yet, i.e. it uses many different concepts, since `signal` package is evolving a lot.

In [ ]:
def discrete_time_system_analysis(b, a, fs=1, N=2**13):
    """plot impulse, step, frequency responses and z-plane of H(z)
    
           b0 z^0 + b1 z^-1 + b2 z^-2 + ... + bN z^-N
    H(z) = ------------------------------------------
           a0 z^0 + a1 z^-1 + a2 z^-2 + ... + aM z^-M
           
    b = [b0, b1, b2, ..., bN]
    a = [a0, a1, a2, ..., aM]
    
    currently len(a)==len(b) must be ensured by appropriate zeropadding
    """
    #still hard coded, TBD:
    #figure size
    #impulse/step response length
    #dlti handling
    #group_delay discontinuities and automatic ylim, yticks is not really useful
 
    if len(a)!=len(b):
        print('signal.dlti requires a and b of same length, please consider zeropadding')
    
    plt.figure(figsize=(12,14))

    df = fs / N  # frequency step between DFT bins and for freqz()
    f = np.arange(0, fs, df)  # frequency vector [0...fs)
    W = 2*np.pi*f/fs  # digital angular frequency [0...2pi)
    [W, H] = signal.freqz(b, a, worN=W, whole='True')
    [W, gd] = signal.group_delay((b,a), w=W, whole='True')  # gd in samples
    zpk = signal.tf2zpk(b, a)
    z = zpk[0]  # get zeros
    p = zpk[1]  # get poles
    k = zpk[2]  # get gain
    sys = signal.dlti(b, a, dt=1)  # handle k as integer, i.e. unit time step
    
    #plot impulse response
    ax1 = plt.subplot(3,2,1)
    [k, h] = signal.dimpulse(sys, n=65)
    h = np.squeeze(h)
    ax1.stem(k, h, basefmt='None')
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax1.grid(True)
    ax1.set_xlabel(r'$k$')
    ax1.set_ylabel(r'impulse response $h[k]$')

    #plot step response
    ax2 = plt.subplot(3,2,2)
    [k, he] = signal.dstep(sys, n=65)
    he = np.squeeze(he)
    ax2.stem(k, he, basefmt='None')
    ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax2.grid(True)
    ax2.set_xlabel(r'$k$')
    ax2.set_ylabel(r'step response $h\epsilon[k]$')

    # plot frequency response magnitude
    ax3 = plt.subplot(3,2,3)
    ax3t = ax3.twiny()

    ax3.plot(W/(2*np.pi), 20*np.log10(np.abs(H)), color='C1', lw=2)  # plotted below grid :-(
    ax3.set_xscale('linear')
    ax3.set_xlim([0, 1])
    ax3.tick_params(axis='x', labelcolor='C1')
    ax3.set_xlabel(r'$\frac{\Omega}{2\pi} = \frac{f}{f_s}$', color='C1')
    ax3.set_xticks(np.arange(0,1.1,0.1))
    ax3.set_ylabel('magnitude in dB   or   20 lg|H| / dB', color='k')
    ax3.grid(True, color='mistyrose', which='major')
    ax3.set_axisbelow(True)

    ax3t.plot(W/(2*np.pi)*fs, 20*np.log10(np.abs(H)), color='C0', lw=2)
    ax3t.set_xscale('log')
    ax3t.set_xlim([f[1], f[-1]])
    ax3t.tick_params(axis='x', labelcolor='C0')
    ax3t.set_xlabel('frequency in Hz   or   f / Hz for $f_s$ = '+'{0:5.2f}'.format(fs)+' Hz', color='C0')
    ax3t.grid(True, color='lavender', which='minor')
    ax3t.set_axisbelow(True)

    # plot frequency response phase
    ax4 = plt.subplot(3,2,4)
    ax4.plot(W, np.angle(H)*180/np.pi)
    ax4.set_xlabel(r'digital angular frequency in radian   or   $\Omega$ / rad')
    ax4.set_ylabel(r'phase in degree   or $\angle$ H / deg')
    ax4.set_xlim([0, 2*np.pi])
    ax4.set_xticks(np.arange(0,2*np.pi+np.pi/4, np.pi/4))
    ax4.set_xticklabels([r'0', r'$\pi/4$', r'$\pi/2$', r'$3/4\pi$', r'$\pi$', r'$5/4\pi$', r'$3/2\pi$', r'$7/4\pi$', r'$2\pi$'])
    ax4.set_ylim([-180,+180])
    ax4.set_yticks(np.arange(-180,+180+45,45))
    ax4.grid(True)

    # zplane from scratch
    # note the dirty handling with counting zeros/poles and indicating them with
    # text annotations in the plot with increasing font size
    # this will be a mess for large number of zeros/poles
    # however, we use this to somehow safely indicate multiple poles at the same
    # location
    # at the moment there is no handling of zplane in signal, matplotlib, pyplot
    # one might use https://gist.github.com/endolith/4625838#file-zplane-py
    ax5 = plt.subplot(3,2,5)
    c = 1
    for ztmp in z:
        ax5.plot(np.real(ztmp), np.imag(ztmp),'o', fillstyle='none', markersize = 10, color='C0')
        ax5.text(np.real(ztmp), np.imag(ztmp), c, color='C0', fontsize=6+2*c)
        c += 1
    c = 1    
    for ptmp in p:
        ax5.plot(np.real(ptmp), np.imag(ptmp),'x', fillstyle='none', markersize = 10, color='C0')
        ax5.text(np.real(ptmp), np.imag(ptmp), c, color='C0', fontsize=6+2*c)
        c += 1
    ax5.set_facecolor('gainsboro')  # set whole background to light gray
    if p.size > 0:  # check if we have poles at all
        r_ROC = np.max(np.abs(p))
        circ = patches.Circle([0,0], radius=r_ROC, color='w')  # non-ROC circle
        ax5.add_patch(circ)  # cut out non-ROC region with white        
    phi = np.arange(0,2*np.pi,2*np.pi/360)
    ax5.plot(np.cos(phi), np.sin(phi), 'k-')  # plot unit circle
    ax5.grid(True)
    ax5.set_aspect('equal')
    ax5.set_title('z-plane: poles x, zeros o, ROC in grey')
    ax5.set_xlabel(r'real part $\Re(z)$')
    ax5.set_ylabel(r'imaginary part $\Im(z)$')
    
    # plot frequency response group delay
    ax6 = plt.subplot(3,2,6)
    ax6.plot(W/np.pi, gd/fs)
    ax6.set_xscale('linear')
    ax6.set_xlim([0, 2])
    ax6.set_xlabel(r'$\frac{\Omega}{\pi}$')
    ax6.set_ylabel(r'group delay in seconds   or   $\tau_\mathrm{GD}$ / s')
    ax6.grid(True, which='both')

We demonstrate frequency axis handling with

* logarithmic x-axis along f / Hz for magnitude, top
* linear x-axis along $\frac{\Omega}{2\pi} = \frac{f}{f_s}$ for magnitude, bottom 
* linear x-axis along $\Omega$ for phase
* linear x-axis along $\frac{\Omega}{\pi}$ for group delay

in the subplots below.
We should get familiar with these different styles, since they often occur and have pros/cons in different applications / visualization strategies.
The choice of which response maps to which handling is rather arbitrary and we can freely switch to whatever style is best suitable.

We discuss systems that were analytically evaluated in the tutorials. Feel free to play around with other systems that you find in textbooks, online material or that come up by your own.

## Exercise 7.3: System H1

In [ ]:
# FIR filter, finite impulse response vector h would be handled as
b = [1, 1, -1, 1/2]  # = h
a = [1, 0, 0, 0]  # len(a)==len(b) handling :-(
discrete_time_system_analysis(b, a, fs=1)
plt.savefig('System_UE7_3_H1.png')

## Exercise 7.3: System H3

In [ ]:
# IIR filter 
b = [2, 0, 1]
a = [1, -1/2, 0]  # note the sign reversal of coef a1 compared to block diagram
discrete_time_system_analysis(b, a, fs=1)
plt.savefig('System_UE7_3_H3.png')

## Exercise 8.1

In [ ]:
# IIR filter
b = [+1, -1, +2]
a = [+1, -1/2, +1/4]  # note the sign reversal of coefs a1, a2 compared to block diagram
discrete_time_system_analysis(b, a, fs=1)
plt.savefig('system_UE8_1.png')

## Discrete-Time Version of the Laplace-Domain ODE Example

The analog lowpass filter is transfered to discrete-time domain by setting the sampling frequency 100 times above the cut-frequency and using the so called bilinear transform.

The 2nd order differential equation leads to a 2nd order difference equation, which can be interpreted as a 2nd order recursive filter, usually referred to as IIR filter.

An obvious difference.

In [ ]:
# digital filter design with so called bilinear transform of a 
# continuous-time ODE (we will learn this in detail in the DSP course):
# ODE RLC-example from 'solving_2nd_order_ode.pdf' /
# 'frequency_response_2nd_order_ode.pdf' is
# 16/25 y''(t) + 24/25 y'(t) + y(t) = DiracDelta(t), y'(t=0)=0, y(t=0)=0
# for example sampled with
fs = np.ceil((5/4)/(2*np.pi)*100)  # sampling frequency in Hz, note: omega0 = 5/4
# note that we just round up to integer for nicer plotting
print('fs = ',fs, 'Hz')
[b, a] = signal.bilinear([25/16], [1, 24/16, 25/16], fs=fs)
print('b = ', b)
print('a = ', a)
discrete_time_system_analysis(b, a, fs=fs)
#we obtain a discrete-time 2nd order IIR-filter with lowpass characteristics
plt.savefig('AnalogODE_Bilinear.png')

In [ ]:
# pass through
b = [+1]
a = [+1]
discrete_time_system_analysis(b, a, fs=1)
plt.savefig('PassThru.png')